In [5]:
import json
import pandas as pd
import numpy as np
from pandas import DataFrame

AIR_DENSITY=1.225
TURBINE_EFF=0.4
SOLAR_PANEL_EFF=0.18
PERF_RATE=0.75
GAMMA=0.0045
WIND_CITIES=["Izmir","Canakkale","Balikesir","Manisa"]

In [6]:
#Deneme dosyası oldğu için çalışmaz!
with open ("Konya_weather.json","r") as f:
    data = json.load(f)
df=pd.json_normalize(data,record_path=['days','hours'],meta=[['days','datetime']])


FileNotFoundError: [Errno 2] No such file or directory: 'Konya_weather.json'

In [48]:
target_colunms=['days.datetime','datetime','temp','solarradiation','windspeed']
df_final=df[target_colunms].copy()

,days.datetime,datetime,temp,solarradiation,windspeed
0,2025-08-15,00:00:00,77.0,0.0,5.8
1,2025-08-15,01:00:00,75.2,0.0,6.9
2,2025-08-15,02:00:00,77.0,0.0,4.7
3,2025-08-15,03:00:00,75.2,0.0,1.1
4,2025-08-15,04:00:00,73.4,0.0,8.1
...,...,...,...,...,...
2227,2025-11-15,19:00:00,48.1,0.0,6.9
2228,2025-11-15,20:00:00,44.5,0.0,5.8
2229,2025-11-15,21:00:00,44.5,0.0,6.9
2230,2025-11-15,22:00:00,44.5,0.0,3.4


In [54]:
def convert_units(df1: DataFrame) -> DataFrame:
    #F TO C
    df1['temp_c']=(df1['temp']-32)*5/9
    #MPH TO M/S
    df1['windspeed_ms']=df1['windspeed']*0.44704
    #merging datatime day-hour
    df1['Full Date']=pd.to_datetime(df1['days.datetime']+' - '+df1['datetime'])
    deleted_columns=['days.datetime','datetime','windspeed','temp']
    df1.drop(deleted_columns,inplace=True,axis=1)
    return df1
df_final=convert_units(df_final)

KeyError: 'temp'

In [55]:
df_final

,solarradiation,temp_c,windspeed_ms,Full Date
0,0.0,25.000000,2.592832,2025-08-15 00:00:00
1,0.0,24.000000,3.084576,2025-08-15 01:00:00
2,0.0,25.000000,2.101088,2025-08-15 02:00:00
3,0.0,24.000000,0.491744,2025-08-15 03:00:00
4,0.0,23.000000,3.621024,2025-08-15 04:00:00
...,...,...,...,...
2227,0.0,8.944444,3.084576,2025-11-15 19:00:00
2228,0.0,6.944444,2.592832,2025-11-15 20:00:00
2229,0.0,6.944444,3.084576,2025-11-15 21:00:00
2230,0.0,6.944444,1.519936,2025-11-15 22:00:00


In [ ]:
#Matematical Calculations
#RAW WIND ENERGY
def raw_wind_energy_cal(df: DataFrame) -> None:
    vel=df['windspeed_ms']
    calculation_part=(vel**3)*AIR_DENSITY*TURBINE_EFF
    df['RAW WIND ENERGY']=np.where(((vel>3)&(vel<25)),calculation_part,0)
def raw_solar_energy_cal(df: DataFrame) -> None:
    df['RAW SOLAR ENERGY'] = df['solarradiation'] * SOLAR_PANEL_EFF * PERF_RATE * (1 - GAMMA * (df['temp_c'] - 25))
raw_wind_energy_cal(df_final)
raw_solar_energy_cal(df_final)
df_final